In [9]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras

housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data,housing.target)
X_train,X_valid,y_train,y_valid = train_test_split(X_train_full,y_train_full)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_train_A,X_train_B = X_train[:,:5],X_train[:,2:]
X_valid_A,X_valid_B = X_valid[:,:5],X_valid[:,2:]
X_test_A,X_test_B = X_test[:,:5],X_test[:,2:]
X_new_A,X_new_B = X_test_A[:3],X_test_B[:3]
X_valid = scaler.fit_transform(X_valid)
X_test = scaler.fit_transform(X_test)

In [10]:
input_ = keras.layers.Input(shape = X_train.shape[1:])
hidden1 = keras.layers.Dense(30,activation="relu")(input_)
hidden2 = keras.layers.Dense(30,activation="relu")(hidden1)
concat = keras.layers.Concatenate()([input_,hidden2])
output = keras.layers.Dense(1)(concat)
model = keras.Model(inputs = [input_],outputs = [output])


In [13]:
class WideAndDeepModel(keras.Model):
    def __init__(self,units=30,activation="relu",**kwargs):
        super().__init__(**kwargs)
        self.hidden1 = keras.layers.Dense(units,activation=activation)
        self.hidden2 = keras.layers.Dense(units,activation=activation)
        self.main_output = keras.layers.Dense(1)
        self.aux_output = keras.layers.Dense(1)
    def call(self,inputs):
        input_A,input_B = inputs
        hidden1 = self.hidden1(input_B)
        hidden2 = self.hidden2(hidden1)
        concat = keras.layers.concatenate([input_A,hidden2])
        main_output = self.main_output(concat)
        aux_output = self.aux_output(hidden2)
        return main_output,aux_output
model = WideAndDeepModel()

In [31]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience = 10,restore_best_weights=True)
checkpoint_cb = keras.callbacks.ModelCheckpoint("../Models/2_TF_functional_SubclassedCalif")

In [32]:
model.compile(loss=["mse","mse"],loss_weights=[0.9,.1],optimizer="sgd")
history = model.fit([X_train_A,X_train_B],[y_train,y_train],
                    epochs = 40,
                    validation_data=([X_valid_A,X_valid_B],[y_valid,y_valid]),
                    callbacks=[checkpoint_cb,early_stopping_cb])
y_pred_main,y_pred_aux = model.predict([X_new_A,X_new_B])
print(y_pred_main)

Epoch 1/40
363/363 [==============================] - 0s 1ms/step - loss: 0.3206 - output_1_loss: 0.3124 - output_2_loss: 0.3939 - val_loss: 52971.7656 - val_output_1_loss: 53581.4609 - val_output_2_loss: 47484.4727
Epoch 2/40
363/363 [==============================] - 0s 1ms/step - loss: 0.3213 - output_1_loss: 0.3133 - output_2_loss: 0.3938 - val_loss: 9880.0156 - val_output_1_loss: 9045.5205 - val_output_2_loss: 17390.5078
Epoch 3/40
363/363 [==============================] - 0s 1ms/step - loss: 0.3218 - output_1_loss: 0.3139 - output_2_loss: 0.3931 - val_loss: 15773.5342 - val_output_1_loss: 15303.6973 - val_output_2_loss: 20002.0781
Epoch 4/40
363/363 [==============================] - 0s 1ms/step - loss: 0.3195 - output_1_loss: 0.3116 - output_2_loss: 0.3913 - val_loss: 9887.8516 - val_output_1_loss: 9002.4648 - val_output_2_loss: 17856.2969
Epoch 5/40
363/363 [==============================] - 0s 1ms/step - loss: 0.3188 - output_1_loss: 0.3110 - output_2_loss: 0.3886 - val_loss:

In [30]:
total_loss,main_loss,aux_loss=model.evaluate([X_test_A,X_test_B],[y_test,y_test])
model.save("../Models/2_TF_functional_SubclassedCalif")


162/162 [==============================] - 0s 740us/step - loss: 7351.3682 - output_1_loss: 6678.3564 - output_2_loss: 13408.4775
INFO:tensorflow:Assets written to: ../Models/2_TF_functional_SubclassedCalif/assets


In [27]:
model = keras.models.load_model("../Models/2_TF_functional_SubclassedCalif")